In [0]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() \
                      else "cpu")
#device = "cpu"
print("working on: " + str(device))

working on: cuda


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path="drive/My Drive/AML/"
df = pd.read_csv(path + "pmsm_temperature_data.csv")

In [0]:
df_sep = [df[df.profile_id==profile].drop(['profile_id','torque'], axis=1).reset_index(drop=True) for profile in df.profile_id.unique()]

train_min = df[~df.profile_id.isin([4, 8, 16, 24, 32, 40, 48, 51, 35, 42])].drop(['profile_id','torque'], axis=1).min()
train_max = df[~df.profile_id.isin([4, 8, 16, 24, 32, 40, 48, 51, 35, 42])].drop(['profile_id','torque'], axis=1).max()

df_train = [(df_sep[i] - train_min)/(train_max - train_min)\
            for i in list(range(0,52)) if i not in [4, 8, 16, 24, 32, 
                                                    40, 48, 51, #val
                                                    35, 42]] #test

df_val = [(df_sep[i] - train_min)/(train_max - train_min)\
          for i in [4, 8, 16, 24, 32, 40, 48, 51]]

df_test = [(df_sep[i] - train_min)/(train_max - train_min) for i in [35, 42]]

In [0]:
def dataloader(data, length, FEATURES, TARGET, shuffle=True, out=1):
    while True:
    # genera una lista di (i_serie, i_obs)
        tuples = [[(df_i, i) for i, x in enumerate(data[df_i]) if i >= length + out]
                  for df_i, _ in enumerate(data)]
        tuples = sum(tuples, [])  # flattenizza
        # shuffle
        if shuffle:
            np.random.shuffle(tuples)

        # yielda le osservazioni
        for df_i, i in tuples:
            X_lagged = data[df_i][(i - length - out):(i - out), FEATURES + TARGET]
            y = data[df_i][(i-out):(i), TARGET]
            yield X_lagged, y

In [0]:
def outputSize(in_size, kernel_size, stride, padding):
  output = int((in_size - kernel_size + 2*(padding)) / stride) + 1
  return(output)

class CNN_Net_2(nn.Module):
    def __init__(self, batch, in_c, out, 
                 filtro1, filtro2,
                 num_ser, 
                 neuroni1, neuroni2,
                 kernel1, kernel2, kernel3, kernel4, 
                 #padding1, padding2, padding3, padding4, 
                 stride1, stride2, stride3, stride4):
        super(CNN_Net_2, self).__init__()
        self.batch_size = 1
        self.in_c = in_c
        l0 = num_ser
        l1 = outputSize(l0,kernel1,stride1,0)
        l2 = outputSize(l1,kernel2,stride2,0)
        l3 = outputSize(l2,kernel3,stride3,0)
        l4 = outputSize(l3,kernel4,stride4,0)
        #print(l4)

        self.conv1 = nn.Conv1d(in_channels= in_c, 
                               out_channels= filtro1, 
                               kernel_size= kernel1,
                               stride= stride1,
                               padding=0)    
        self.pool1 = nn.MaxPool1d(kernel2,stride=stride2,padding=0) 
        self.conv2 = nn.Conv1d(filtro1,filtro2,kernel3,stride=stride3,padding=0)
        self.pool2 = nn.MaxPool1d(kernel4,stride=stride4,padding=0)
        self.fc1 = nn.Linear(filtro2*l4, neuroni1)
        self.fc2 = nn.Linear(neuroni1, neuroni2)
        self.fc3 = nn.Linear(neuroni2, out)

    def forward(self, x):
        batch_size, _, _ = x.shape
        x =  self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(batch_size, self.num_flat_features(x))
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        return  self.fc3(x)

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:       # Get the products
            num_features *= s
        return num_features

In [0]:
best_mse = torch.load(path + "models/best_model_cnn").to(device)
best_mse_stator = torch.load(path + "models/best_model_cnn_stator").to(device)

In [0]:
def check_performance(data_array, batch_size, look_back):
    FEATURES = [0, 1, 2, 3, 4, 5, 6]
    TARGET = [7]

    gen = dataloader([df.values for df in data_array], look_back, FEATURES, TARGET, shuffle=False)
    size = sum(df.shape[0] for df in data_array)

    y_test = []
    y_pred_all = []
    for x in range(0, size, batch_size):
        X_test = []
        for i in range(batch_size):
            try:
                X,y = next(gen)
                X_test.append(X)
                y_test.append(y)
            except:
                break
        inpt = np.array(X_test).reshape(-1, look_back, 8)
        x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
        y_pred = best_mse(x_test_batch)

        y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())


    pm_err = (y_pred_all - np.array(y_test).reshape(-1))**2

    FEATURES = [0, 1, 2, 3, 4, 5, 6]
    TARGET = [8, 9, 10]

    torch.cuda.empty_cache()

    gen = dataloader([df.values for df in data_array], look_back, FEATURES, TARGET, shuffle=False)

    y_test = []
    y_pred_all = []
    for x in range(0, size, batch_size):
        X_test = []
        for i in range(batch_size):
            try:
                X,y = next(gen)
                X_test.append(X)
                y_test.append(y)
            except:
                break
        inpt = np.array(X_test).reshape(-1, look_back, 10)
        x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
        y_pred = best_mse_stator(x_test_batch)

        y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())


    pm_err_stat = (y_pred_all.reshape(-1) - np.array(y_test).reshape(-1))**2


    return np.mean(np.concatenate((pm_err, pm_err_stat)))

In [10]:
look_back = 60
batch_size = 18803

check_performance(df_train, batch_size, look_back)

2.3169168124802886e-05

In [0]:
#validation

In [11]:
look_back = 60
batch_size = 7403

check_performance(df_val, batch_size, look_back)

2.281504908293559e-05

In [0]:
#test

In [12]:
look_back = 60
batch_size = 28439

check_performance(df_test, batch_size, look_back)

2.3870405136875294e-05

# LSTM

In [0]:
class LSTM_Net(nn.Module):
    def __init__(self, features, hidden, h1, h2, out):
        super(LSTM_Net, self).__init__()
        self.h1 = h1
        self.h2 = h2

        self.hidden_size = hidden
        self.features = features

        self.lstm = nn.LSTM(self.features, self.hidden_size, 1, #feature_size, hidden_size, num_layer
                            batch_first = True) 
        self.fc1 = nn.Linear(self.hidden_size, 
                                h1)
        self.out = nn.Linear(h1, out)

        if self.h2 != 0:
            self.fc2 = nn.Linear(h1, h2)
            self.out = nn.Linear(h2, out)
        
        #we do it stateless so there is no need for the hidden_state
        #self.hidden = None #torch.randn(1, ??, self.hidden) #num_layer, batch, hidden_size

    def forward(self, x):
        batch_size, _, _ = x.shape 
        
        x, _ =  self.lstm(x)
        x = F.tanh(x[:,-1].view(batch_size, -1))
        
        x = F.relu(self.fc1(x))
        if self.h2 != 0:
            x = F.relu(self.fc2(x))
        return self.out(x)

In [15]:
best_mse = torch.load(path + "models/best_model_lstm").to(device)
best_mse_stator = torch.load(path + "models/best_model_lstm_stator").to(device)

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [16]:
look_back = 60
batch_size = 18803

check_performance(df_train, batch_size, look_back)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


9.83681378898177e-06

In [17]:
look_back = 60
batch_size = 7403

check_performance(df_val, batch_size, look_back)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


9.715135123805541e-06

In [18]:
look_back = 60
batch_size = 28439

check_performance(df_test, batch_size, look_back)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


1.0090488300309295e-05